In [1]:
from pyspark.sql import SparkSession
import findspark
import os, sys

findspark.init()

# Descrição do desafio

O objetivo é realizar o processamento de uma grande base de dados utilizando o Apache Spark e a série histórica de preços de combustíveis no Brasil de acordo com a Agência Nacional de Gás Natural e Biocombustíveis (ANP). A base deve ser carregada no Apache Spark, explorada e pré-processada para depois insights serem extraídos. Algumas visualização também podem ser geradas no Matplotlib ou Seaborn.

Fonte: https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

O repositório disponibiliza os dados em arquivos CSV organizados por semestre, compreendendo todo o período a partir do ano de 2004.

## Colunas

- Regiao - Sigla
- Estado - Sigla
- Municipio
- Revenda
- CNPJ da Revenda
- Nome da Rua
- Numero Rua
- Complemento
- Bairro
- Cep
- Produto
- Data da Coleta
- Valor de Venda
- Valor de Compra
- Unidade de Medida
- Bandeira


# Definição de pastas e caminhos

In [2]:
# Definição do caminho das pastas raíz e data
root = os.getcwd()

data_folder = os.path.join(root, 'data')

# Configuração do PySpark

In [3]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

# Leitura dos dados

In [4]:
list_csv = [os.path.join(data_folder,file) for file in os.listdir(data_folder) if file.endswith('.csv')]

print(f'Número de arquivos: {len(list_csv)}')

Número de arquivos: 39


In [5]:
raw_data = spark.read.csv(list_csv, inferSchema=True, header=True, sep=';')

# Análise de dados exploratória

In [6]:
raw_data.show()

+--------------+--------------+-------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+--------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|    Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep| Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+-------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+--------+--------------+--------------+---------------+-----------------+--------------------+
|            CO|            MS| CAMPO GRANDE|AUTO POSTO APAREC...| 86.807.609/0001-92|       RUA BRILHANTE|     2,787|            NULL|VILA BANDEIRANTES|79006-560|GASOLINA|    01/07/2004|          2,71|         2,0888|       R

In [7]:
print(f'N columns: {len(raw_data.columns)}')
print(f'N lines: {raw_data.count()}')

N columns: 16
N lines: 22645940


In [8]:
raw_data.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



Algumas colunas numéricas estão declaradas como string, o que dificultará a análise dessas informações do ponto de vista de extrair estatíticas descritivas e criar gráficos.

Portanto, deve-se fazer a conversão destas colunas: **Valor de Venda** e **Valor de Compra**.

Além disso, é importante transformar a coluna **Data da Coleta** para o tipo datetime.

Entretanto, antes será necessário realizar uma limpeza na base de dados. Visualizando os .csvs originais foi identificado que várias linhas da base estão com seus valores deslocados em relação às posições das colunas.

Em resumo, algumas linhas estão, por exemplo, com o valor de **Data da Coleta** na coluna **Valor de Venda**, o número de **Valor de Venda** na coluna **Valor de Compra** e assim por diante.

Uma das alternativas para tratar essa inconsistência é descartar as linhas que estão com esse problema, já que trata-se de um montante pequeno com relação à base como um todo.

# Pré-processamento

In [9]:
raw_data.createOrReplaceTempView('raw_data_sql')

In [19]:
unidade_unique = spark.sql('SELECT DISTINCT `Unidade de Medida` FROM raw_data_sql')

In [20]:
unidade_unique.show()

+-----------------+
|Unidade de Medida|
+-----------------+
|       R$ / litro|
|          R$ / m³|
|          R$ / m�|
|             NULL|
+-----------------+

